In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import os
import time

# --- Configuración Kaggle ---
os.environ['KAGGLE_CONFIG_DIR'] = "."

# --- Descargar y descomprimir datos ---
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip -o udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

# --- Cargar datasets ---
train = pd.read_csv("train.csv", na_values=["", " ", "NO SABE", "SIN INFORMACION", "NS/NR"])
test = pd.read_csv("test.csv", na_values=["", " ", "NO SABE", "SIN INFORMACION", "NS/NR"])

# --- Limpieza y renombrado de columnas ---
train.columns = train.columns.str.strip().str.replace(" ", "_").str.upper()
test.columns = test.columns.str.strip().str.replace(" ", "_").str.upper()

# --- Rellenar valores faltantes ---
for col in train.select_dtypes(include=[np.number]).columns:
    med = train[col].median()
    train[col] = train[col].fillna(med)
    if col in test.columns:
        test[col] = test[col].fillna(med)

for col in train.select_dtypes(include="object").columns:
    train[col] = train[col].fillna("DESCONOCIDO")
    if col in test.columns:
        test[col] = test[col].fillna("DESCONOCIDO")

# --- Feature Engineering ---
for df in [train, test]:
    df['PROG_VALOR'] = df['E_PRGM_ACADEMICO'] + "_" + df['E_VALORMATRICULAUNIVERSIDAD']
    df['PROG_DEPT'] = df['E_PRGM_ACADEMICO'] + "_" + df['E_PRGM_DEPARTAMENTO']

# --- Variable objetivo ---
target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
train['RENDIMIENTO_GLOBAL'] = train['RENDIMIENTO_GLOBAL'].replace(target_map)

# --- Separar X / y ---
X = train.drop(columns=["ID", "RENDIMIENTO_GLOBAL"])
y = train["RENDIMIENTO_GLOBAL"]

# --- Convertir categóricas a 'category' ---
for col in X.select_dtypes(include="object").columns:
    X[col] = X[col].astype('category')
    test[col] = test[col].astype('category')

# Número de folds para validación cruzada
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Número de clases del problema
n_classes = len(target_map)
preds_test = np.zeros((len(test), n_classes, n_splits))

start_time = time.time()

# --- Validación cruzada StratifiedKFold ---
for i, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\nFold {i+1}/{n_splits}")

    # Dividir datos en entrenamiento y validación para el fold
    xtr, xval = X.iloc[train_idx], X.iloc[val_idx]
    ytr, yval = y.iloc[train_idx], y.iloc[val_idx]

# Modelo LightGBM con hiperparámetros avanzados
    model = LGBMClassifier(
        n_estimators=1500,
        learning_rate=0.02,
        max_depth=14,
        num_leaves=200,
        subsample=0.85,
        colsample_bytree=0.7,
        reg_alpha=2,
        reg_lambda=4,
        min_child_samples=25,
        class_weight='balanced',
        random_state=42+i,
        verbose=50
    )

# Entrenamiento con early stopping
    model.fit(
        xtr,
        ytr,
        eval_set=[(xval, yval)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=100)]
    )

# Predicción en validación para medir accuracy por fold
    val_pred = model.predict(xval)
    acc = accuracy_score(yval, val_pred)
    print(f"Accuracy fold {i+1}: {acc:.4f}")

    preds_test[:, :, i] = model.predict_proba(test.drop(columns=["ID"]))

final_pred = np.argmax(preds_test.mean(axis=2), axis=1)

inv_target_map = {v: k for k, v in target_map.items()}
pred_labels = [inv_target_map[p] for p in final_pred]

# --- Crear archivo de submission ---
submission = pd.DataFrame({
    "ID": test["ID"],
    "RENDIMIENTO_GLOBAL": pred_labels
})

submission.to_csv("submission_ensemble_lgbm.csv", index=False)
print(f"\nArchivo submission_ensemble_lgbm.csv generado correctamente en {time.time()-start_time:.2f} segundos.")



udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


/tmp/ipython-input-1225817095.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['RENDIMIENTO_GLOBAL'] = train['RENDIMIENTO_GLOBAL'].replace(target_map)


Streaming output truncated to the last 5000 lines.
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 14
[LightGBM] 